In [203]:
import pandas as pd

df = pd.read_csv('code/8daysstats.csv')
df = df[df['state']=='SUCCEEDED']
print(df.columns, len(df))

Index(['state', 'submitTime', 'startTime', 'finishTime', 'queueTime',
       'runTime', 'NumMaps', 'HDFS_INPUT_SIZE', 'HDFS_OUTPUT_SIZE',
       'MAP_CPU_USAGE_MSEC', 'REDUCE_CPU_USAGE_MSEC', 'MAP_MEM_USAGE_B',
       'REDUCE_MEM_USAGE_B', 'HIVE_RECORDS_IN', 'HIVE_RECORDS_OUT',
       'HIVE_RECORDS_INTERMEDIATE', 'SLOTS_MILLIS_MAPS',
       'SLOTS_MILLIS_REDUCES', 'TOTAL_LAUNCHED_MAPS', 'TOTAL_LAUNCHED_REDUCES',
       'DATA_LOCAL_MAPS', 'RACK_LOCAL_MAPS', 'MILLIS_MAPS', 'MILLIS_REDUCES',
       'VCORES_MILLIS_MAPS', 'VCORES_MILLIS_REDUCES', 'MB_MILLIS_MAPS',
       'MB_MILLIS_REDUCES', 'PHMAP_MEM_USAGE_B', 'PHREDUCE_MEM_USAGE_B',
       'PHPHYSICAL_MEMORY_B', 'jobid', 'query', 'outputdir', 'scratchdir',
       'sessionid', 'query.id', 'local.scratchdir', 'user.name', 'job',
       'inputdir', 'workflow.id', 'workflow.dag', 'workflow.node'],
      dtype='object') 98585


In [84]:
# groupby workflow.id
wfsize_df = df.groupby('workflow.id').size().reset_index(name='counts')
# is there anyones who has the same workflow id 
wf_gdf = df.groupby('workflow.id')

query = wf_gdf.get_group(wfsize_df.iloc[wfsize_df['counts'].idxmax()]['workflow.id']).iloc[0]['query']
username = wf_gdf.get_group(wfsize_df.iloc[wfsize_df['counts'].idxmax()]['workflow.id']).iloc[0]['user.name']
dag = wf_gdf.get_group(wfsize_df.iloc[wfsize_df['counts'].idxmax()]['workflow.id'])['workflow.dag'].iloc[0]
inputsize = wf_gdf.get_group(wfsize_df.iloc[wfsize_df['counts'].idxmax()]['workflow.id'])[['workflow.node', 'runTime', 'HDFS_INPUT_SIZE']]
print(username, '\n', query)

mphani 
 

Create table guestdb.sg_grid_sum_p as
Select * from
(
select a.*,b.sys_serial_no from(
select distinct g.ASUP_ID,s.PATH,g.ATTR,g.NAME,G.VALUE,'G' as TAB_NAME from smfdbv2.grid_configurations  g, asuprep.summary_attributes  s
where g.attr in ('GIDV','GPVD','GSIV','GCMP','GENC','GHSH') and g.ASUP_id = s.ASUP_ID and s.oid like "%.1.1"
and s.oid not like "%.1.1.1")a left join default.system b on a.asup_id = b.asup_id

  union all
select a.*,b.sys_serial_no from(
select distinct il.ASUP_ID,s.PATH,il.ATTR,il.NAME,il.VALUE,'I' as TAB_NAME from asuprep.ilm_xml  il, asuprep.summary_attributes  s
where il.ASUP_id = s.ASUP_ID and s.oid like "%.1.1"
and s.oid not like "%.1.1.1")a left join default.system b on a.asup_id = b.asup_id

union all

select a.*,b.sys_serial_no from(
  select  ASUP_ID,
        PATH,
        ATTR,
        NAME,
        VALUE,
  'S' as TAB_NAME
from asuprep.summary_attributes
where attr in ('XECT','PSCA','PSCU','XASC','XEWC','XISC','XSNI','XUSC','XSRX','XSTX','XWR

In [201]:
print(df[df['jobid'] == 'job_1531656020138_353795']['inputdir'].values[0])
print(df[df['jobid'] == 'job_1531656020138_353795']['HDFS_INPUT_SIZE'].values[0], 5202*20)
print(df[df['jobid'] == 'job_1531656020138_353795']['NumMaps'].values[0])

256000000/(1<<20)
33554432

hdfs://datalake-nnha/hive_external_tables/ES_ASUP/dt=20180805
9532802 104040
20


33554432

In [190]:
import datetime

def input_stats(inputs, input_size, NumMaps):
    inputs_table = {}
    for x in inputs:
        input_name = x.replace('hdfs://datalake-nnha/', '')
        if input_name.startswith('hive_external_tables/'):
            input_name = input_name.replace('hive_external_tables/', '')
            if 'dt' in input_name:
                table = input_name.split('/dt=')[0]
                date = input_name.split('/dt=')[1]
                if table not in inputs_table:
                    inputs_table[table] = []
                inputs_table[table].append(date)
            else: 
                table = input_name.split('/')[0]
                att = input_name.replace(table, '') if len(input_name.replace(table, '')) > 0 else 'att'  
                if table not in inputs_table:
                    inputs_table[table] = []
                inputs_table[table].append(att)

        elif '-mr-' in input_name:
            idep = input_name.split('-mr-')[0]
            mrdep = '-mr-' + input_name.split('-mr-')[1].split('/')[0]
            depdir = idep + mrdep
            mrout = input_name.split('-mr-')[1].split('/')[0][1]
            if depdir not in inputs_table:
                inputs_table[depdir] = []
            inputs_table[depdir].append(mrout)

    print('input size:', input_size//(1<<20), '(MB), num of files', len(inputs), 'n mappers', NumMaps)
    for x in inputs_table:
        print(len(inputs_table[x]), 'of', x)

for index, row in df[df['user.name'] == 'mphani'].iterrows():
    print('\n............................\n', row['workflow.id'], ',' , row['workflow.node'], '\n', 
         datetime.datetime.fromtimestamp(row['startTime']//1000), 
         row['query'].replace('\n', '').replace('\t', '')[:12],  row['jobid'], '\n outputdir', row['outputdir'])
    input_stats(row['inputdir'].split(','), row['HDFS_INPUT_SIZE'], row['NumMaps'])



............................
 hive_mphani_20180812035555_cedc0baa-6346-418c-b38b-9bd8b6c9d9aa , Stage-1 
 2018-08-12 10:55:36 CREATE TABLE job_1531656020138_355937 
 outputdir /tmp/hive/mphani/205b2fa3-beae-46c1-94ed-e842adad3b7f
input size: 139069 (MB), num of files 895 n mappers 561
1 of asuprep.db
47 of tmp/hive/mphani/205b2fa3-beae-46c1-94ed-e842adad3b7f/hive_2018-08-12_03-55-11_441_402844525801747308-1/-mr-10004
847 of SYSTEM

............................
 hive_mphani_20180812035555_cedc0baa-6346-418c-b38b-9bd8b6c9d9aa , Stage-2 
 2018-08-12 10:56:18 CREATE TABLE job_1531656020138_355951 
 outputdir /tmp/hive/mphani/205b2fa3-beae-46c1-94ed-e842adad3b7f
input size: 2 (MB), num of files 1 n mappers 90
1 of tmp/hive/mphani/205b2fa3-beae-46c1-94ed-e842adad3b7f/hive_2018-08-12_03-55-11_441_402844525801747308-1/-mr-10003

............................
 hive_mphani_20180812035656_8e592be7-7a3f-4b48-90dc-6217e340a417 , Stage-15 
 2018-08-12 10:57:55 Create table job_1531656020138_355977 


 2018-08-13 07:14:07 insert into  job_1531656020138_368506 
 outputdir /tmp/hive/mphani/7742154d-f9d6-4103-8268-475e155af586
input size: 2042 (MB), num of files 1 n mappers 91
1 of tmp/hive/mphani/7742154d-f9d6-4103-8268-475e155af586/hive_2018-08-13_00-07-10_585_3045922618721874783-1/-mr-10002

............................
 hive_hive_20180813001414_e799c7cd-79f5-437f-94da-c1355be78f87 , Stage-1 
 2018-08-13 07:14:35 select max(a job_1531656020138_368508 
 outputdir /tmp/hive/mphani/148c92a2-8e5b-4c24-b672-f01892d4eced
input size: 14821 (MB), num of files 1 n mappers 84
1 of bartdb

............................
 hive_hive_20180813001717_aad352d9-26e0-48e7-ae27-76a91a4677f0 , Stage-1 
 2018-08-13 07:17:36 select max(p job_1531656020138_368521 
 outputdir /tmp/hive/mphani/148c92a2-8e5b-4c24-b672-f01892d4eced
input size: 28826 (MB), num of files 1 n mappers 127
1 of bartdb

............................
 hive_mphani_20180813012323_d703ed3a-4eae-41a6-b682-5fb1950b853c , Stage-1 
 2018-08-13 

 hive_mphani_20180813035757_a4bda5f9-56d3-46c3-aa8f-1eab68681515 , Stage-15 
 2018-08-13 10:58:39 Create table job_1531656020138_370451 
 outputdir /tmp/hive/mphani/2abf3a78-977b-4cfd-860a-1c8ac5348e76
input size: 161 (MB), num of files 1 n mappers 71
1 of asuprep.db

............................
 hive_mphani_20180813035757_a4bda5f9-56d3-46c3-aa8f-1eab68681515 , Stage-21 
 2018-08-13 10:59:14 Create table job_1531656020138_370463 
 outputdir /tmp/hive/mphani/2abf3a78-977b-4cfd-860a-1c8ac5348e76
input size: 161 (MB), num of files 1 n mappers 71
1 of asuprep.db

............................
 hive_mphani_20180813035757_a4bda5f9-56d3-46c3-aa8f-1eab68681515 , Stage-13 
 2018-08-13 11:01:40 Create table job_1531656020138_370499 
 outputdir /tmp/hive/mphani/2abf3a78-977b-4cfd-860a-1c8ac5348e76
input size: 139232 (MB), num of files 896 n mappers 563
1 of asuprep.db
47 of tmp/hive/mphani/2abf3a78-977b-4cfd-860a-1c8ac5348e76/hive_2018-08-13_03-57-51_371_3763560813796408165-1/-mr-10040
848 of SYS

 hive_mphani_20180816035757_a373c738-7c81-4889-8c63-d027251b159e , Stage-21 
 2018-08-16 10:59:45 Create table job_1531656020138_416875 
 outputdir /tmp/hive/mphani/9f89f80e-c9cf-4600-ad4f-581d474d31a2
input size: 161 (MB), num of files 1 n mappers 71
1 of asuprep.db

............................
 hive_mphani_20180816035757_a373c738-7c81-4889-8c63-d027251b159e , Stage-13 
 2018-08-16 11:03:08 Create table job_1531656020138_416916 
 outputdir /tmp/hive/mphani/9f89f80e-c9cf-4600-ad4f-581d474d31a2
input size: 139757 (MB), num of files 899 n mappers 566
1 of asuprep.db
47 of tmp/hive/mphani/9f89f80e-c9cf-4600-ad4f-581d474d31a2/hive_2018-08-16_03-57-51_800_4563851371140364599-1/-mr-10040
851 of SYSTEM

............................
 hive_mphani_20180816035757_a373c738-7c81-4889-8c63-d027251b159e , Stage-2 
 2018-08-16 11:04:22 Create table job_1531656020138_416933 
 outputdir /tmp/hive/mphani/9f89f80e-c9cf-4600-ad4f-581d474d31a2
input size: 175 (MB), num of files 1 n mappers 71
1 of asuprep.

input size: 139910 (MB), num of files 901 n mappers 507
1 of tmp/hive/mphani/0f691cdf-ff00-42ec-a009-599c7ab4e1a1/hive_2018-08-18_03-57-49_396_7936475535389380932-1/-mr-10010
47 of tmp/hive/mphani/0f691cdf-ff00-42ec-a009-599c7ab4e1a1/hive_2018-08-18_03-57-49_396_7936475535389380932-1/-mr-10046
853 of SYSTEM

............................
 hive_mphani_20180818035757_28533d34-c24c-4581-9606-db1516cbaba5 , Stage-35 
 2018-08-18 11:04:56 Create table job_1534545750476_3509 
 outputdir /tmp/hive/mphani/0f691cdf-ff00-42ec-a009-599c7ab4e1a1
input size: 163 (MB), num of files 1 n mappers 70
1 of asuprep.db

............................
 hive_mphani_20180818035757_28533d34-c24c-4581-9606-db1516cbaba5 , Stage-3 
 2018-08-18 11:05:21 Create table job_1534545750476_3517 
 outputdir /tmp/hive/mphani/0f691cdf-ff00-42ec-a009-599c7ab4e1a1
input size: 139912 (MB), num of files 901 n mappers 507
1 of tmp/hive/mphani/0f691cdf-ff00-42ec-a009-599c7ab4e1a1/hive_2018-08-18_03-57-49_396_7936475535389380932-1/-

In [180]:
58431/(256)

228.24609375

In [82]:
import graph_tool.all as gt

lb_vid = {}
g = gt.Graph(directed=True)
estrl = dag.split(',')
vlbl = g.new_vertex_property("string")  

for es in estrl:
    src = es.split('>')[0]
    if src not in lb_vid:
        svid = g.add_vertex()
        lb_vid[src] = svid
        vlbl[svid] = src.replace('Stage-', '')
    if len(es.split('>')) == 1:
        continue
        
    dst = es.split('>')[1]
    if dst not in lb_vid:
        dvid = g.add_vertex()
        lb_vid[dst] = dvid
        vlbl[dvid] = dst.replace('Stage-', '')
    sv = lb_vid[src]
    dv = lb_vid[dst]
    
    g.add_edge(sv, dv)

g.vertex_properties["label"] = vlbl
print(len(lb_vid))
gt.graph_draw(g, vertex_text=g.vp.label, vertex_font_size=12, vertex_size=3,
           output_size=(1000, 1000), output="graph-nodes.png")


35


<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f86e59f2438, at 0x7f86edf1eeb8>